In [ ]:
import numpy as np
import icecream as ic
from tqdm.auto import tqdm

## Problem 0 - Setup
Description: generate a small instance (10 items, 3 knapsacks, 2 dimensions).
Goal: provide a minimal example to verify evaluation and neighborhood functions.

In [3]:
# Problem 0:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

## Problem 1 - Medium instance
Description: 20 items, 3 knapsacks, 2 dimensions.
Box: Experimental goal - run the memetic algorithm and report the best value found.

In [16]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

## Problem 2 - Large instance
Description: 100 items, 10 knapsacks, 10 dimensions.
Box: Experimental goal - run the memetic algorithm and collect best values to compare with SA.

In [18]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

## Problem 3 - Very large instance
Description: 5000 items, 100 knapsacks, 100 dimensions.
Box: Performance note - generating valid solutions can be expensive; consider alternative approaches (pure SA, starting from invalid solutions, lighter initialization).

In [20]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [6]:
def is_valid(solution):
    # Check that each item is assigned to at most one knapsack
    if not np.all(solution.sum(axis=0) <= 1):
        return False

    # For each knapsack, compute the total weight and verify capacity constraints
    for k in range(NUM_KNAPSACKS):
        items_in_knapsack_k = solution[k]
        weight_of_knapsack_k = WEIGHTS[items_in_knapsack_k].sum(axis=0)
        if not np.all(weight_of_knapsack_k <= CONSTRAINTS):
            return False

    return True

In [7]:
def evaluate(solution):
    # Return -1 if the solution is invalid, otherwise the sum of selected item values
    if not is_valid(solution):
        return -1.0
    items_placed = np.any(solution, axis=0)
    total_value = VALUES[items_placed].sum()
    return float(total_value)

In [8]:
def move(solution):
    # Generate a neighbor by moving (or removing) a random item
    neighbor = solution.copy()
    item_to_move = rng.integers(0, NUM_ITEMS)
    new_knapsack_idx = rng.integers(-1, NUM_KNAPSACKS)
    neighbor[:, item_to_move] = False
    if new_knapsack_idx != -1:
        neighbor[new_knapsack_idx, item_to_move] = True

    return neighbor

In [9]:
def hill_climber_fast(initial_solution: np.ndarray, max_steps_no_improvement: int):
    # Simple local search: accepts only improving moves and stops when no improvement is found
    current_solution, current_score = initial_solution, evaluate(initial_solution)
    steps_without_improvement = 0
    while steps_without_improvement < max_steps_no_improvement:
        neighbor = move(current_solution)
        neighbor_score = evaluate(neighbor)
        if neighbor_score > current_score:
            current_solution, current_score = neighbor, neighbor_score
            steps_without_improvement = 0
        else:
            steps_without_improvement += 1
    return current_solution, current_score

In [10]:
def create_random_valid_solution():
    # Build a valid solution by attempting to insert items in random order
    solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
    shuffled_items = list(range(NUM_ITEMS))
    rng.shuffle(shuffled_items)
    for item_idx in shuffled_items:
        knapsack_idx = rng.integers(0, NUM_KNAPSACKS)
        solution[knapsack_idx, item_idx] = True
        if not is_valid(solution):
            solution[knapsack_idx, item_idx] = False
            
    return solution

In [11]:
def crossover(parent1: np.ndarray, parent2: np.ndarray) -> np.ndarray:
    # Recombination operator: for each item inherit assignment from one of the parents
    child = np.zeros_like(parent1)
    for i in range(NUM_ITEMS):
        if rng.random() < 0.5:
            child[:, i] = parent1[:, i]
        else:
            child[:, i] = parent2[:, i]
    return child

In [12]:
def simulated_annealing_fast(initial_solution: np.ndarray, max_steps: int, initial_temp: float, cooling_rate: float):
    """
    Fast version of Simulated Annealing used as a local search (memetic phase).
    It accepts worsening moves with decreasing probability to escape local minima.
    """
    current_solution = initial_solution
    current_score = evaluate(current_solution)
    best_solution, best_score = current_solution, current_score
    temperature = initial_temp
    
    for _ in range(max_steps):
        neighbor = move(current_solution)
        neighbor_score = evaluate(neighbor)
        
        # accept always if better, otherwise with probability depending on the temperature
        if neighbor_score >= current_score or rng.random() < np.exp((neighbor_score - current_score) / temperature):
            current_solution, current_score = neighbor, neighbor_score
        
        if current_score > best_score:
            best_solution, best_score = current_solution, current_score
            
        temperature *= cooling_rate

    return best_solution, best_score

In [13]:
def memetic_algorithm(generations: int, mu: int, lambda_: int):
    # Initialize a population of feasible solutions
    population = [create_random_valid_solution() for _ in range(mu)]
    # For problem 3 you may initialize differently to speed up evaluations
    # population = [np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool) for _ in range(mu)]
    best_solution_so_far, best_score_so_far = None, -1
    for gen in tqdm(range(generations)):
        offspring = []
        for _ in range(lambda_):
            # random parent selection and recombination
            parent1 = population[rng.integers(0, mu)]
            parent2 = population[rng.integers(0, mu)]
            child = crossover(parent1, parent2)
            # small random mutation
            child = move(child)
            # local phase: improve the individual with fast SA
            improved_child, _ = simulated_annealing_fast(child, max_steps=75, initial_temp=10.0, cooling_rate=0.99)
            offspring.append(improved_child)

        # selection: merge parents and offspring, then keep the best mu individuals
        combined_population = population + offspring
        scores = [evaluate(ind) for ind in combined_population]
        sorted_indices = np.argsort(scores)[::-1]
        population = [combined_population[i] for i in sorted_indices[:mu]]
        
        current_best_score = scores[sorted_indices[0]]
        if current_best_score > best_score_so_far:
            best_score_so_far = current_best_score
            best_solution_so_far = population[0]
            print(f"  Generation {gen+1}: new record! value = {best_score_so_far}")

    return best_solution_so_far, best_score_so_far

In [ ]:

GENERATIONS = 50
POPULATION_SIZE = 20  # mu (population size)
OFFSPRING_SIZE = 200 # lambda (number of offspring per generation)

best_solution, best_score = memetic_algorithm(
    generations=GENERATIONS, 
    mu=POPULATION_SIZE, 
    lambda_=OFFSPRING_SIZE
)

if best_score <= 0:
    print("Valid solution not found")
else:
    print("Value:", best_score)

### Solution Problem 1
Run the memetic algorithm on problem 1 instance and print the best value found.

In [17]:
# PROBLEM 1 - run using memetic algorithm parameters similar to the example
GENERATIONS = 50
POPULATION_SIZE = 20
OFFSPRING_SIZE = 200
best_solution, best_score = memetic_algorithm(
    generations=GENERATIONS, mu=POPULATION_SIZE, lambda_=OFFSPRING_SIZE
)
if best_score <= 0:
    print("no solution found")
else:
    print("Value:", best_score)

  2%|▏         | 1/50 [00:00<00:17,  2.86it/s]

  Generation 1: new record! value = 823.0


  4%|▍         | 2/50 [00:00<00:15,  3.00it/s]

  Generation 2: new record! value = 838.0


  8%|▊         | 4/50 [00:01<00:15,  2.98it/s]

  Generation 4: new record! value = 847.0


 10%|█         | 5/50 [00:01<00:15,  2.99it/s]

  Generation 5: new record! value = 880.0


100%|██████████| 50/50 [00:17<00:00,  2.85it/s]

Value: 880.0


### Solution Problem 2
Run the memetic algorithm on problem 2 instance and print the best value found.

In [19]:
# PROBLEM 2 - run
GENERATIONS = 50
POPULATION_SIZE = 20
OFFSPRING_SIZE = 200
best_solution, best_score = memetic_algorithm(
    generations=GENERATIONS, mu=POPULATION_SIZE, lambda_=OFFSPRING_SIZE
)
if best_score <= 0:
    print("no solution found")
else:
    print("Value:", best_score)

  2%|▏         | 1/50 [00:00<00:17,  2.88it/s]

  Generation 1: new record! value = 19708.0


 24%|██▍       | 12/50 [00:04<00:12,  2.96it/s]

  Generation 12: new record! value = 19767.0


 26%|██▌       | 13/50 [00:04<00:12,  2.94it/s]

...

  Generation 48: new record! value = 23757.0


 98%|█████████▊| 49/50 [00:18<00:00,  2.51it/s]

  Generation 49: new record! value = 24007.0


100%|██████████| 50/50 [00:18<00:00,  2.71it/s]

Value: 24007.0


### Solution Problem 3
For the very large instance we run a single SA execution starting from a valid solution to get a quick estimate.

In [ ]:
# PROBLEM 3 - simulated annealing starting from a random valid solution
#result with 10 seconds run SA
initial_solution = create_random_valid_solution()
best_solution, best_score = simulated_annealing_fast(initial_solution, max_steps=1000, initial_temp=100.0, cooling_rate=0.995)
if best_score <= 0:
    print("no solution found")
else:
    print("Value:", best_score)

Value: 801343.0


## Final observations
- For small and medium problems the memetic algorithm significantly improves results compared to pure SA.
- For problem 3 generating valid solutions is expensive; hybrid approaches or lighter initializations are recommended.

In [ ]:
# Shortened run for Problem 3: use a lightweight local search to get a quick estimate
current_solution = create_random_valid_solution()
current_score = evaluate(current_solution)
_, current_score = simulated_annealing_fast(current_solution, 5000, 10, 0.99)

print(current_score)
